In [8]:
3053541818 + 2939275739 * 54//6

29507023469

In [3]:
sum([695279662, 389386780, 980092719, -151885171, 37619092, 988782657])

2939275739

In [5]:
sum([695279662, 389386780, 980092719, 37619092, 988782657])

3091160910

In [ ]:
 [0, 988782657, 1684062319, 2073449099, 3053541818, 3091160910]